In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
import sklearn.linear_model
import sklearn.model_selection
import random

random.seed(137)
rest = random.random()

def weight(word):
    # overfitted
    if word == 'lerxst@wam.umd.edu':
        return 100.0
    if word == 'car':
        return random.random()
    if word == 'dog':
        return - random.random()
    return random.random()

def has(word, text):
    return word in text 

def feature(index):
    return 1

# Applied Machine Learning

## Linear Models

### Recap

- We have some dataset
- We identify the problem and define the loss function
- Then we minimize the total loss (empirical risk, or objective) using available (training) data
- We vary parameters to minimize the objective function
- The minimizing parameters are then used to predict unknown values

### A text classification problem

Lets consider the **20 newsgroups** dataset:

In [22]:
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups()
text, label = data['data'][0], data['target_names'][data['target'][0]]
print(label)
print('----')
print(text[:300])

rec.autos
----
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be 


### A linear model for classification

Let us consider a function that tells if the `text` comes from `rec.autos`

In [3]:
weight('car')*has('car', text) + weight('and')*has('and', text) + rest

1.4727164112007762

Alternatively say `car` is `0` and `dog` is `1`:

In [4]:
weight(0)*feature(0) + weight(1)*feature(1) + rest

0.6520663252475314

How do we find those `weight` ($w$) for all the words?

### Gradient Descent

- Last time we used `opt.fmin` and it magically found the solution
- The method is simple though
- Start with random weights $w_0$
- Iterate: $w_{i+1} = w_{i} - \alpha \times \nabla \mathsf{objective}(w_i)$
- Approximate gradient: $\nabla f(x) \sim \frac{f(x+\epsilon) - f(x-\epsilon)}{2\epsilon}$
- All we need to know is the gradient of objective

### Gradient of loss

- Last time we considered a regression problem and used $(y-p)^2$
- The gradient w.r.t $p$ is obvious: $- 2 (y - p)$

### Gradient check

How can we ensure the gradient is correct?

In [5]:
def loss(y, p):
    return (y-p)**2

def gradient(y, p):
    return -2*(y-p)

p = 0.1
y = 0.3
eps = 0.001
gradient(y, p), (loss(y, p+eps) - loss(y, p-eps)) / (2*eps)

(-0.39999999999999997, -0.400000000000001)

### Gradient descent in code

In [29]:
current_p = random.random()


print('Initial', current_p)
alpha = 0.3 # learning rate
for i in range(10):
    current_p = current_p - alpha*gradient(y, current_p)
    print(i, current_p)

Initial 0.42717884123187067
0 0.35087153649274827
1 0.3203486145970993
2 0.30813944583883973
3 0.3032557783355359
4 0.30130231133421437
5 0.30052092453368573
6 0.30020836981347426
7 0.3000833479253897
8 0.3000333391701559
9 0.3000133356680624


### Classification loss

- We will use something called **logistic loss**

In [36]:
def loss(y, p):
    return np.log2(1.0 + np.exp(-y*p))
    
loss(+1, 100.0), loss(+1, -100.0)

(0.0, 144.26950408889635)

### Logistic Regression in sklearn

In [39]:
model = sklearn.linear_model.SGDClassifier(loss='log')
# feature 1 = car
# feature 2 = 3d
# feature 3 = and
# feature 4 = with
example_1 = [1, 0, 1, 1]
label_1 = [1] # rec.autos
example_2 = [0, 1, 1, 1]
label_2 = [0] # comp.graphics
model.fit([example_1, example_2], np.ravel([label_1, label_2]))
model.coef_ # weights

array([[ 19.16157655, -19.59912539,  -0.43754883,  -0.43754883]])

### Overfitting

- We can always come up with a model that fits data perfectly

In [9]:
weight('lerxst@wam.umd.edu')

100.0

- For some reason that's not what we want. Why?
- First, we need to measure if such a thing happens

### Splitting the data

- Obviously we should not test what we fit against
- We should fit (train) the model on some part of data
- Next, we check the model against the rest

### Leave-one-out

- Generate as many samples as there are examples
- Gives you a good estimate if you don't have a lot of data
- Gets impractical on huge datasets

In [10]:
loo = sklearn.model_selection.LeaveOneOut()
for train, test in loo.split([1,2,3,4,5]):
    print(train, test)

[1 2 3 4] [0]
[0 2 3 4] [1]
[0 1 3 4] [2]
[0 1 2 4] [3]
[0 1 2 3] [4]


### Cross validation

- Split the dataset into a few (say 5) non-overlapping parts
- Four parts go to training data and one part goes to test data
- Do the above 5 times to train the model and test it
- 5 runs (precision): 0.9, 0.89, 0.91, 0.92, 0.88 -> mean 0.9 
- Makes a decent way to *detect* overfitting

### Cross validation in sklearn

Let's consider indices of data

In [51]:
xval = sklearn.model_selection.KFold(n_splits=3, shuffle=True, random_state=9)
for train, test in xval.split([1,2,3,4,5,6]):
    print(train, test)

[0 2 3 4] [1 5]
[0 1 4 5] [2 3]
[1 2 3 5] [0 4]


### This thing is an ill-posed problem

- A mathematical problem is ill-posed when the solution is not unique
- That's exactly the case of regression/classification/...
- We need to make the problem well-posed: *regularization*

### Structural risk minimization

- Structural risk is empirical risk plus regularizer
- Instead of minimizing empirical risk we find some tradeoff
- Regularizer is a function of model we get
- $\mathsf{objective} = \mathsf{loss} + \mathsf{regularizer}$

### Regularizer

- A functions that reflects the complexity of a model
- What is the complexity of a set of 'if ... then'?
- Not obvious for linear model but easy to invent something

### $\ell_1$ regularizer

- $\ell_1: \| w \|_1 = |w(car)| + |w(dog)| + ...$
- Derivative is const
- Forces weight to be zero if it doesn't hurt performance much 
- Use if you believe some features are useless

In [12]:
classification_model = sklearn.linear_model.SGDClassifier(loss='log', penalty='l1');
regression_model = sklearn.linear_model.SGDRegressor(penalty='l1');

### $\ell_2$ regularizer

- $\ell_2: \| w \|_2 = w(car)^2 + w(dog)^2 + ...$
- Derivative is linear
- Forces weights to get *similar* magnitude if it doesn't hurt performance much
- Use if you believe all features are more or less important

In [13]:
classification_model = sklearn.linear_model.SGDClassifier(loss='log', penalty='l2');
regression_model = sklearn.linear_model.SGDRegressor(penalty='l2');

### Elastic net

- Just a weighted sum of $\ell_1$ and $\ell_2$ regularizers
- An attempt to get useful properties of both

In [14]:
regression_model = sklearn.linear_model.SGDRegressor(penalty='elasticnet')

### Limitations of linearity

- In low-dimensional spaces linear models are not very 'powerful' (can we define that?)
- The higher dimensionality, the more powerful linear model becomes

### Sparse features

- We say features are sparse when most of the values are zero
- Examples: visited hosts, movies that user liked, ...
- Sparse features are efficient in high-dimensional setting

### One hot encoding, hashing trick

- One way to encode categorical things like visited websites
- We enumerate all the websites
- We put 1 to position of every host, 0 otherwise
- Hashing trick: instead of enumerating them just hash

In [15]:
print(hash('the')) # int64
print(hash('the') % 2**16, 2**16)

-2132726271299405475
14685 65536


### Hashing vectorizer in sklearn

In [16]:
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = HashingVectorizer(n_features=1000000000, binary=True, norm=None)
features = vectorizer.fit_transform(['hello there', 'Hey there'])
print(features)

w = [4.3, 9999999999 more of them]
w[613153351]*1.0 + w[753185237]*1.0

  (0, 613153351)	1.0
  (0, 753185237)	1.0
  (1, 318325784)	1.0
  (1, 753185237)	1.0


### When do we use linear models?

- It is definitely the first thing to try if you have some text data
- In general a good choice for any sparse data
- This approach is pretty much the fastest one
- Even if some method outperforms, you still get a good baseline

### Self-assessment questions

1. You noticed that your linear model learned a weight of **95.3** for the word `the`. *Is there a problem? Y/N*
2. The train loss is **0.43** and the test loss is **0.39**. *Is it an example of ..? a) overfitting b) underfitting c) I don't know*
3. You've got basically infinite amounts of data. *Do you have to use regularization? Y/N*
4. You believe your dataset is pretty noisy and some features are broken. *You use a) L1 b) L2 c) no regularization* 
5. Why do we hash words? *a) it's simpler b) it's faster c) it's more reliable*

### Homework 1

- Load dataset, create linear model, train, and explain the results
- The template is provided: `HSE-AML-HW1.ipynb`
- Hint: check the code examples for `KFold`, `HashingVectorizer`, `LogisticRegression`